In [ ]:
# Read unfiltered water-level data; compare with model\
import glob, os
import pandas as pd
import numpy as np
import xarray as xr
import xroms
import matplotlib.pyplot as plt
from scipy import stats
import hvplot.xarray

# These files were downloaded from https://stn.wim.usgs.gov/fev/ on 1/19/2023
data_dir = r'/vortexfs1/home/csherwood/proj/NOPP_Michael/data/'
hwm_file = 'Michael_FilteredHWMs.csv'
peaks_file = 'Michael_FilteredPeaks.csv'

# url='http://geoport.whoi.edu/thredds/dodsC/vortexfs1/usgs/Projects/Michael2018/michael20/michael20.ncml'
url = 'http://geoport.whoi.edu/thredds/dodsC/vortexfs1/usgs/Projects/Michael2018/michael33/qck/michael_qck.ncml'
run_name = 'm33'
# url = 'http://geoport.whoi.edu/thredds/dodsC/vortexfs1/usgs/Projects/Michael2018/michael37/qck/michael_qck.ncml'
# run_name = 'm37'

# Based on datum for 8729108 Panama City, FL
MSLtoNAVD88 = -0.11 # (m) Add this offset to convert model output (MSL) to gauge output (NAVD88)

# Peak water levels are here
dstart = '2018-10-10 12:00'
dend = '2018-10-11 00:00'

### Read in the model output

In [ ]:
dsm=xroms.open_netcdf(url)
dsm

In [ ]:
# Get the maximum indices of the grid...
max_eta = dsm['eta_rho'][-1].values
max_xi = dsm['xi_rho'][-1].values

### Read in the observations
#### List of the unfiltered datafiles

In [ ]:
file_list = glob.glob(data_dir+'*unfiltered.nc')
print(file_list)

In [ ]:
# These had different filenames
file_list2 = glob.glob(data_dir+'*csv.nc')
print(file_list2)

#### Loop through all of the files and pull out ids and lat/lon
(Some longitudes are positive...fix that).

In [ ]:
all_lats = []
all_lons = []
all_ids = []
i=0
for f in file_list+file_list2:
    ds = xr.open_dataset(f)
    all_ids.append( ds.attrs['stn_station_number'] )
    all_lats.append( ds['latitude'].values )
    # treat positive longitudes (wtf?)
    lont = ds['longitude'].values
    if lont > 0: lont = np.array(-lont)
    all_lons.append( lont )

### Cheapo plot of max. water levels

In [ ]:
# Cheapo plot
# find the maximum water level over the entire time
# adding the .compute() function was key to speeding up the lat/lon lookup below
plt_name = '/vortexfs1/home/csherwood/proj/NOPP_Michael/figs/'+run_name+'_unfiltered_map.png'
zmax = dsm.zeta.max(dim='ocean_time').compute()
ax = zmax.plot(x="lon_rho", y="lat_rho")
plt.plot(all_lons, all_lats, 'ok')
for i, id in enumerate(all_ids):
    plt.text(all_lons[i]+.02,all_lats[i]+.02,id,fontsize=7)
plt.savefig( plt_name, dpi=200, bbox_inches='tight')

### Plot comparison between obs. and model

In [ ]:
# hr = 'unfiltered_water_surface_height_above_reference_datum'
# hf = 'water_surface_height_above_reference_datum'
lats = []
lons = []
ids = []
i=0
for f in file_list+file_list2:
    ds = xr.open_dataset(f)
    id = ds.attrs['stn_station_number']
    lat = ds['latitude'].values
    # treat positive longitudes (wtf?)
    lon = ds['longitude'].values
    if lon > 0: lon = np.array(-lon)

    # get indices in model domain. Values of zero or max_eta, max_xi mean location is not in model grid
    ix, iy = dsm.Hwave.xroms.argsel2d( lon, lat )
    in_domain = True
    if ix==0 or ix >= max_eta or iy==0 or iy >= max_xi:
        in_domain = False
        
    if in_domain:
        ids.append( id )
        lats.append( lat )
        lons.append( lon )
        print('\n{}'.format(id))
        print('lat, lon: {:.3f},{:.3f}'.format(lat, lon))
        print('inst. elev. {:.3f},{:.3f}'.format( ds.attrs['sensor_orifice_elevation_at_deployment_time'], ds.attrs['sensor_orifice_elevation_at_retrieval_time']))
        print('Sampling interval: ',ds.attrs['time_coverage_resolution'])
        # observations
        tobs = ds['time'].sel(time=slice(dstart,dend)).values
        hraw = ds['unfiltered_water_surface_height_above_reference_datum'].sel(time=slice(dstart,dend)).values
        hfil = ds['water_surface_height_above_reference_datum'].sel(time=slice(dstart,dend)).values
        # model data
        tmod = dsm.ocean_time.sel(ocean_time=slice(dstart,dend)).values
        zeta = dsm.zeta.sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).values + MSLtoNAVD88
        hwave = dsm.Hwave.sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).values
        zhplus = zeta+0.5*hwave
        zhminus = zeta-0.5*hwave
        
        imax = np.nanargmax(hfil)
        peak_obs = hfil[imax]
        peak_time = tobs[imax]
        # print('hmax, time: ',peak_obs,peak_time)
        ts1 = r'Obs. peak {:.2f} at {}'.format(peak_obs, str(peak_time)[:16])
        
        jmax = np.nanargmax(zeta)
        peak_mod = zeta[jmax]
        peak_mod_time = tmod[jmax]
        # print('mod. max, time: ',peak_mod,peak_mod_time)
        ts2 = r'Mod. peak {:.2f} at {}'.format(peak_mod,str(peak_mod_time)[:16])

        
        if True:
            plt_name = '/vortexfs1/home/csherwood/proj/NOPP_Michael/figs/'+run_name+'_unfiltered_comparisons_'+id+'.png'
            fig, ax = plt.subplots(ncols=1)
            plt.plot(tobs, hraw, c='dimgray',label='raw obs.')
            plt.plot(tobs, hfil, c='black',label='filtered obs.')
            plt.plot(tmod, zeta, c='blue',label='model zeta')
            plt.plot(tmod, zhplus, '--r', label='waves')
            plt.plot(tmod, zhminus, '--r' )
            plt.ylabel('Elevation (m NAVD88)')
            plt.text(x=1.05, y=.95, s=ts1, fontsize=10, c='k',transform=ax.transAxes,\
             horizontalalignment='left', verticalalignment='bottom')
            plt.text(x=1.05, y=.9, s=ts2, fontsize=10, c='b',transform=ax.transAxes,\
             horizontalalignment='left', verticalalignment='bottom')
            plt.xlabel('2018')
            plt.title(id)
            ax.legend(loc='upper right', bbox_to_anchor=(1, 1))
            plt.savefig(plt_name,dpi=200,bbox_inches='tight')
    

#### Example of how to select in time at one point

In [ ]:
zeta = dsm.zeta.sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).values

In [ ]:
zeta

#### Exploring one of the unfiltered datasets

In [ ]:
ds = xr.open_dataset(file_list[0])
ds

In [ ]:
ds.attrs['stn_station_number']
ds.attrs['sensor_orifice_elevation_at_deployment_time']
ds.attrs['sensor_orifice_elevation_at_retrieval_time']
ds.attrs['sensor_orifice_elevation_units']
ds.attrs['initial_land_surface_elevation']
ds.attrs['final_land_surface_elevation']

In [ ]:
hr= 'unfiltered_water_surface_height_above_reference_datum'
hf= 'water_surface_height_above_reference_datum'
lat = ds['latitude'].values
lon = ds['longitude'].values
print(lat, lon)

In [ ]:
dsm.Hwave.xroms.argsel2d(lon, lat)

In [ ]:
# what happens if the lon, lat is not in the model domain?
dsm.Hwave.xroms.argsel2d(lon+10, -lat)

In [ ]:
dstart = '2018-10-10 12:00'
dend = '2018-10-11 00:00'

zw = dsm['zeta'].sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).values + dsm['Hwave'].sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).values
t = dsm['ocean_time'].sel(ocean_time=slice(dstart,dend))

fig, ax = plt.subplots(ncols=1)
ds[hr].sel(time=slice(dstart,dend)).plot(c='dimgray',label='raw obs.')
ds[hf].sel(time=slice(dstart,dend)).plot(c='black',label='filtered obs.')
dsm.zeta.sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).plot(c='blue',label='model zeta')
plt.plot(t,zw,'--r',label='model zeta+Hwave')
fig.legend()


In [ ]:
# plotting with hvplot
a=ds[hraw].hvplot(x='time',c='dimgray',label='raw obs.')
b=ds[hf].hvplot(x='time',c='black',label='filtered obs.')
c = dsm.zeta.sel(ocean_time=slice(dstart,dend)).xroms.sel2d(lon, lat).hvplot(x='ocean_time', c='blue',label='model')

a*b*c